In [32]:
%load_ext autoreload
%autoreload 2
import os
import matplotlib.pyplot as plt
import seaborn as sns
from os.path import join
from tqdm import tqdm
import pandas as pd
import joblib
import sys
from copy import deepcopy
import imodels
import sklearn
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.linear_model import ElasticNet, LogisticRegression, Ridge, Lasso, RidgeCV, ElasticNetCV, LinearRegression, LassoCV
from collections import defaultdict
import guidance

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Set up a linear regression problem
Groundtruth is fit to the entire dataset (selected via CV) whereas the other models are fit to a small subset

In [33]:
PARAM_GRID_LINEAR_REGRESSION = [
    {
        "est": [
            RidgeCV(), ElasticNetCV(), LinearRegression(), LassoCV()
        ],
    },
]

# DSETS_CLASSIFICATION = ['pima_diabetes']
# X, y, feature_names = imodels.get_clean_dataset("pima_diabetes")
X, y, feature_names = imodels.get_clean_dataset("california_housing")
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=42, test_size=0.99
)

# preprocess gt data
X = sklearn.preprocessing.StandardScaler().fit_transform(X)
y = sklearn.preprocessing.StandardScaler().fit_transform(y.reshape(-1, 1)).flatten()

# preprocess split data
trans = sklearn.preprocessing.StandardScaler()
X_train = trans.fit_transform(X_train)
X_test = trans.transform(X_test)
transy = sklearn.preprocessing.StandardScaler()
y_train = transy.fit_transform(y_train.reshape(-1, 1)).flatten()
y_test = transy.transform(y_test.reshape(-1, 1)).flatten()
print("shapes", X.shape, y.shape, "nunique",
      np.unique(y).size, '-> train', X_train.shape)


def fit_and_get_feats(X_train, y_train, topk=2):
    m = imodels.AutoInterpretableRegressor(
        # m = imodels.AutoInterpretableClassifier(
        param_grid=PARAM_GRID_LINEAR_REGRESSION, refit=True)
    m.fit(X_train, y_train, cv=3)

    # print("best params", m.est_.best_params_)
    # print("best score", m.est_.best_score_)
    # print("best estimator", m.est_.best_estimator_)
    # print("best estimator params", m.est_.best_estimator_.get_params())
    # print('selected from', m.param_grid)
    df = pd.DataFrame(m.est_.cv_results_).sort_values(
        "rank_test_score").reset_index()
    first_cols = ["rank_test_score", "mean_test_score", "std_test_score"]
    df = df[first_cols +
            [c for c in df.columns if c not in first_cols]].round(3)
    # remove std_ cols
    df = df[[c for c in df.columns if "std_" not in c]]

    # Refit top models with best params
    d = defaultdict(list)
    for i in range(topk):
        params = df.loc[i, 'params']
        clf = m.est_.best_estimator_.set_params(**params)
        clf.fit(X_train, y_train)
        clf = clf.steps[0][1]
        d['model'].append(deepcopy(clf))
        d['train_score'].append(clf.score(X_train, y_train))
        d['test_score'].append(clf.score(X_test, y_test))
        d['coef'].append(clf.coef_)
        d['intercept'].append(clf.intercept_)
    d = pd.DataFrame(d)
    return d


d_gt = fit_and_get_feats(X, y, topk=1)
d_small = fit_and_get_feats(X_train, y_train, topk=2)
d = pd.concat((d_gt, d_small))

fetching california_housing from sklearn
shapes (20640, 8) (20640,) nunique 3842 -> train (206, 8)


### Interpretation
Note: these our coefficients after standardizing the inputs.

In [34]:
out = {'feature_names': feature_names + ['Intercept']}
for i in range(len(d)):
    coef = d.iloc[i].coef.tolist() + \
        [d.iloc[i].intercept.tolist()]
    if i == 0:
        out['GT'] = coef
    else:
        # out[f'{str(d.iloc[i]["model"])[:-4]} ({i})'] = coef
        out[i] = coef

coefs = pd.DataFrame.from_dict(out)
col1 = coefs.columns[1]
vabs = np.max(np.abs(coefs[col1]))

display(d.round(3).drop(
    columns=['coef', 'intercept']))
display(
    coefs
    .sort_values(by=col1)
    .style.background_gradient(
        cmap=sns.diverging_palette(
            20, 220, as_cmap=True, center='dark'),
        vmin=-vabs, vmax=vabs
    )
    .format(precision=2)
)

,model,train_score,test_score
0,LinearRegression(),0.606,0.053
0,LassoCV(),0.713,-11.585
1,RidgeCV(),0.715,-12.144


,feature_names,GT,1,2
6,Latitude,-0.78,-0.60,-0.68
7,Longitude,-0.75,-0.57,-0.65
2,AveRooms,-0.23,-0.14,-0.15
5,AveOccup,-0.03,-0.22,-0.22
4,Population,-0.00,0.04,0.05
8,Intercept,-0.00,-0.00,-0.00
1,HouseAge,0.10,0.08,0.09
3,AveBedrms,0.26,0.00,0.01
0,MedInc,0.72,0.80,0.81


# Let's ask GPT some questions about the models

In [40]:
# connect to a chat model like GPT-4 or Vicuna
gpt4 = guidance.llms.OpenAI("gpt-4-0314")
# vicuna = guidance.llms.transformers.Vicuna("your_path/vicuna_13B", device_map="auto")

program = guidance('''
{{#system~}}
You are a helpful and terse assistant.
{{~/system}}

{{#user~}}
I am going to fit a linear model on a dataset that predicts {{target}} from the following features:
- {{feature_list_str}}
How do you expect each of these features to affect the prediction?
{{~/user}}

{{#assistant~}}
{{gen 'expl_initial' temperature=0 max_tokens=600}}
{{~/assistant}}
                   
{{#user~}}
Here are the coefficients resulting from fitting the linear model after normalization (higher coefficients correspond to higher predict {{target}}):
- {{coef1}}                   
Explain what makes sense and what does not.                
{{~/user}}              
            
{{#assistant~}}
{{gen 'expl_1' temperature=0 max_tokens=600}}
{{~/assistant}}
                   
{{#user~}}
Here are the coefficients resulting from fitting a different linear model after normalization (higher coefficients correspond to higher predict {{target}}):
- {{coef2}}                   
Again, explain what makes sense and what does not.                
{{~/user}}                                 
                                      
{{#assistant~}}
{{gen 'expl_2' temperature=0 max_tokens=600}}
{{~/assistant}}
                                      
{{#user~}}
Between the first linear model and the second linear model, which is better and why?
Start your reply with "The first model is better" or "The second model is better"
{{~/user}}                   

{{#assistant~}}
{{gen 'model_comparison' temperature=0 max_tokens=600}}
{{~/assistant}}

''', llm=gpt4)


coef1 = coefs.loc[:, 1].round(2).tolist()
coef2 = coefs.loc[:, 2].round(2).tolist()

# for the sake of illustraton, let's flip a couple of coefficients
coef2[0] = -coef2[0]
coef2[-1] = -coef2[-1]

ans = program(
    target='Housing prices in California',
    feature_list_str='\n- '.join(feature_names),
    coef1='\n- '.join([f'{f}: {c}' for f, c in zip(feature_names, coef1)]),
    coef2='\n- '.join([f'{f}: {c}' for f, c in zip(feature_names, coef2)]),
)

Stop program 
 {{#system~}} 
You are a helpful and terse assistant.
 {{~/system}} 

 {{#user~}} 
I am going to fit a linear model on a dataset that predicts {{target}} from the following features:
- {{feature_list_str}} 
How do you expect each of these features to affect the prediction?
 {{~/user}} 

 {{#assistant~}} 
 {{gen 'expl_initial' temperature=0 max_tokens=600}} 
 {{~/assistant}} 
 
 {{#user~}} 
Here are the coefficients resulting from fitting the linear model after normalization (higher coefficients correspond to higher predict {{target}} ):
- {{coef1}} 
Explain what makes sense and what does not. 
 {{~/user}} 
 
 {{#assistant~}} 
 {{gen 'expl_1' temperature=0 max_tokens=600}} 
 {{~/assistant}} 
 
 {{#user~}} 
Here are the coefficients resulting from fitting a different linear model after normalization (higher coefficients correspond to higher predict {{target}} ):
- {{coef2}} 
Again, explain what makes sense and what does not. 
 {{~/user}} 
 
 {{#assistant~}} 
 {{gen 'expl_2' temperature=0 max_tokens=600}} 
 {{~/assistant}} 
 
 {{#user~}} 
Between the first linear model and the second linear model, which is better and why?
Start your reply with "The first model is better" or "The second model is better"
 {{~/user}} 

 {{#assistant~}} 
 {{gen 'model_comparison' temperature=0 max_tokens=600}} 
 {{~/assistant}}